In [1]:
import pystokes 
import numpy as np, matplotlib.pyplot as plt
# particle radius, self-propulsion speed, number and fluid viscosity
b, vs, Np, eta = 1.0,2, 15, 0.1

#initialise
r = pystokes.utils.initialCondition(Np)  # initial random distribution of positions
p = np.zeros(3*Np); 
p[2*Np:3*Np] = -0.4   # initial orientation of the colloids


def rhs(rp):
    """
    right hand side of the rigid body motion equation
    rp: is the array of position and orientations of the colloids
    returns the \dot{rp} so that rp can be updated using an integrator
    """
    # assign fresh values at each time step
    r = rp[0:3*Np];   p = rp[3*Np:6*Np]
    norm_p = np.dot(p,p)
    p = p/np.sqrt(norm_p)
    F, v, o,T = np.zeros(3*Np), np.zeros(3*Np), np.zeros(3*Np),np.zeros(3*Np)
    
    force.lennardJonesWall(F, r, lje=0.01, ljr=5, wlje=1.2, wljr=3.4)
    torque.magnetic(T,p,m0=1,Bx=0, By=1, Bz=-0)
    #torque.bottomHeaviness(T,p,bh=1.0)
    rbm.mobilityTT(v, r, F)  
    rbm.mobilityRR(o,r,T)
    #rbm.mobilityTR(v,r,T)
    #rbm.mobilityRT(o,r,F)
    
    l2 = pystokes.utils.irreducibleTensors(2, p,Y0=1)
    l3 = pystokes.utils.irreducibleTensors(3, p)
    v3t = vs*p;
    rbm.propulsionT2s(v,r,l2)
    #rbm.propulsionR2s(o,r,l2)
    #rbm.propulsionR4a(o, r, l3)
    rbm.propulsionT3t(v, r, v);
    v = v+v3t;
    return np.concatenate( (v,o) )

rbm   = pystokes.wallBounded.Rbm(radius=b, particles=Np, viscosity=eta)
force = pystokes.forceFields.Forces(particles=Np)
torque = pystokes.forceFields.Torques(particles=Np)

# simulate the resulting system
Tf, Npts = 120, 50
pystokes.utils.simulate(np.concatenate((r,p)), Tf,Npts,rhs, filename='swarms')

# plot the data at specific time instants
#pystokes.utils.plotConfigs(t=[1,40,60,80,100], ms=80, tau=1, filename='swarms')# -*- coding: utf-8 -*-
from scipy.io import loadmat
result=loadmat('swarms')
traj	  = result['X']

from statistics import mean
py=traj[0:Npts,4*Np:5*Np]
pymean=mean(py[Npts-1])
px=traj[0:Npts,3*Np:4*Np]
pxmean=mean(px[Npts-1])
pz=traj[0:Npts,5*Np:6*Np]
pzmean=mean(pz[Npts-1])
alpha=np.arctan(pymean/pzmean)*180/np.pi


for i in range(Npts):
    dx=traj[i+1,0:Np]-traj[i,0:Np]
    dy=traj[i+1,Np:2*Np]-traj[i,Np:2*Np]
    dz=traj[i+1,2*Np:3*Np]-traj[i,2*Np:3*Np]
    speed=10*np.sqrt(dx**2+dy**2+dz**2)
    #fig,ax=plt.subplots()
    #plt.scatter(traj[i,0:Np],traj[i,Np:2*Np],s=40,c=speed, cmap='inferno')
    #plt.xlim(-50,50)
    #plt.ylim(-50,50)
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.quiver(traj[i,0:Np], traj[i,Np:2*Np], traj[i,2*Np:3*Np], traj[i,3*Np:4*Np], traj[i,4*Np:5*Np], traj[i,5*Np:6*Np],length=5)
    ax.set_xlim(-20,20)
    ax.set_ylim(-20,20)
    ax.set_zlim(0,10)
    plt.show()



ModuleNotFoundError: No module named 'pystokes.interface'